## Setup

In [3]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "petclinic"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [4]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.72s/it]

Number of chunks =  432


## Create a dataframe of all the chunks

In [3]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(64, 3)


,text,source,chunk_id
0,Found 48 source code files: ../blog/springblog...,data_input/springblog/springblog.txt,9280403b73124364a9a51ec0ea88c95a
1,""";this.currentPage=1,this.lastPage=1,this.setO...",data_input/springblog/springblog.txt,d206530d2f04474ab5da631313c16068
2,pages=this.getPages();pages.first&&this.show(p...,data_input/springblog/springblog.txt,16ab83f8a56e4e4e817c30768392d9ee
3,"class='pagination'></ul>""):this.$element,listC...",data_input/springblog/springblog.txt,5f12d324dda0462cbec1e1e8b96daf16
4,"itemContainer=$(""<li></li>""),itemContent=$(""<a...",data_input/springblog/springblog.txt,8e6e3f6024c44e2e94d352316c4b6fe3


## Extract Concepts

In [5]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [6]:
## To regenerate the graph with LLM, set this to True
regenerate = False

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

(160, 5)


,node_1,node_2,edge,chunk_id,count
0,petclinic,src,is a directory,06ee23aea9a34b1e973821ef973d1d89,4
1,petclinic,main,is a directory,06ee23aea9a34b1e973821ef973d1d89,4
2,src,java,is a directory,06ee23aea9a34b1e973821ef973d1d89,4
3,org,springframework,belongs to the package,06ee23aea9a34b1e973821ef973d1d89,4
4,spring,samples,belongs to the package,06ee23aea9a34b1e973821ef973d1d89,4


## Calculating contextual proximity

In [7]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)

### Merge both the dataframes

In [8]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,../spring-petclinic/build/generated/aotsources...,cachemeterbinderprovidersconfiguration__beande...,"cbe21f64f80046068d46ba916e885e58,cbe21f64f8004...",contextual proximity,3
1,/,_pr_isie6,"178b377b9fad4004929ff5394ca0846a,178b377b9fad4...",is a global function for browser detection in ...,8
2,/,pr,"178b377b9fad4004929ff5394ca0846a,178b377b9fad4...",is an object that contains functions for prett...,12
3,/,prettyprint,"178b377b9fad4004929ff5394ca0846a,178b377b9fad4...",contextual proximity,4
4,/,prettyprintone,"178b377b9fad4004929ff5394ca0846a,178b377b9fad4...",contextual proximity,4
...,...,...,...,...,...
1066,welcomecontroller.java,system,"06ee23aea9a34b1e973821ef973d1d89,06ee23aea9a34...",contextual proximity,3
1067,welcomecontroller.java,test,"06ee23aea9a34b1e973821ef973d1d89,06ee23aea9a34...",contextual proximity,2
1068,welcomecontroller.java,vet,"06ee23aea9a34b1e973821ef973d1d89,06ee23aea9a34...",contextual proximity,5
1069,window['pr'],/,"178b377b9fad4004929ff5394ca0846a,178b377b9fad4...","is assigned to the PR object, indicating that ...",8


## Calculate the NetworkX Graph

In [9]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(202,)

In [10]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [11]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  17
[['../spring-petclinic/build/generated/aotsources/org/springframework/boot/actuate/autoconfigure/metrics/repository/repositorymetricsproperties__beandefinitions.java', 'cachemeterbinderprovidersconfiguration__beandefinitions.java', 'cachemetricsautoconfiguration__beandefinitions.java', 'cachemetricsregistrarconfiguration__beandefinitions.java', 'metricsendpointautoconfiguration__beandefinitions.java', 'metricsproperties__beandefinitions.java', 'repositorymetricsautoconfiguration__beandefinitions.java', 'simplemetricsexportautoconfiguration__beandefinitions.java', 'simpleproperties__beandefinitions.java', 'systemmetricsautoconfiguration__beandefinitions.java'], ['/', '_pr_isie6', 'pr', 'prettyprint', 'prettyprintone', "window['pr']"], ['actuate', 'compositemeterregistryautoconfiguration', 'datasourcepoolmetricsautoconfiguration', 'heapdumpwebendpointautoconfiguration', 'integrationmetricsautoconfiguration', 'jvmmetricsautoconfiguration', 'logbackmetricsautoco

### Create a dataframe for community colors

In [12]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,../spring-petclinic/build/generated/aotsources...,#578ddb,1
1,cachemeterbinderprovidersconfiguration__beande...,#578ddb,1
2,cachemetricsautoconfiguration__beandefinitions...,#578ddb,1
3,cachemetricsregistrarconfiguration__beandefini...,#578ddb,1
4,metricsendpointautoconfiguration__beandefiniti...,#578ddb,1
...,...,...,...
197,transactionautoconfiguration__beandefinitions....,#db577e,16
198,transactionmanagercustomizationautoconfigurati...,#db577e,16
199,transactionproperties__beandefinitions.java,#db577e,16
200,webendpointautoconfiguration,#9ddb57,17


### Add colors to the graph

In [13]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [14]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
